In [1]:
import pandas as pd
df = pd.read_csv('/Users/riadsouza/Downloads/mimic_keyspace_data.csv')

df

,subject_id,hadm_id,icd9_code,age,gender,admission_type,insurance,ethnicity,hospital_expire_flag
0,41976,173269,3891,65.800000,M,EMERGENCY,Medicare,HISPANIC/LATINO - PUERTO RICAN,0
1,41976,172082,966,65.076712,M,EMERGENCY,Medicare,HISPANIC/LATINO - PUERTO RICAN,0
2,41976,149469,3897,64.830137,M,EMERGENCY,Medicare,HISPANIC/LATINO - PUERTO RICAN,0
3,10093,165393,3891,87.438356,M,EMERGENCY,Medicare,WHITE,1
4,41976,176016,3893,65.076712,M,EMERGENCY,Medicare,HISPANIC/LATINO - PUERTO RICAN,0
...,...,...,...,...,...,...,...,...,...
1305,41976,125449,9604,63.906849,M,EMERGENCY,Medicare,HISPANIC/LATINO - PUERTO RICAN,0
1306,41976,173269,9671,65.468493,M,EMERGENCY,Medicare,HISPANIC/LATINO - PUERTO RICAN,0
1307,10088,149044,966,77.890411,M,EMERGENCY,Private,WHITE,0
1308,10106,133283,8841,63.786301,M,EMERGENCY,Private,WHITE,0


1. What is the top procedure performed for the female patients?

>>> CREATE TABLE IF NOT EXISTS "RiaDSouza_de300lab4".patient_by_gender (
        subject_id int, 
        hadm_id int,
        icd9_code int,
        age float,
        gender text,
        admission_type text,
        insurance text,
        ethnicity text,
        hospital_expire_flag int,
        PRIMARY KEY ((gender, icd9_code), subject_id)
    );
    
>>> INSERT INTO "RiaDSouza_de300lab4".patient_by_gender(subject_id, hadm_id, icd9_code, age, gender, admission_type, insurance, ethnicity, hospital_expire_flag)
VALUES (41976, 173269, 3891, 65.800000, 'M',	'EMERGENCY',	'Medicare',	'HISPANIC/LATINO - PUERTO RICAN',	0);

I would create the table this way since we are filtering based on gender, and therefore gender must be the partition key. Partitioning on gender will also allow for even nodes. We also want icd9_code to be a partition key, as we will be grouping by that as well. We include subject_id as a clustering column since it is unique for each patient.


2. What is the top procedure performed for the medicare and medicaid insurance holders?

>>> CREATE TABLE IF NOT EXISTS "RiaDSouza_de300lab4".patient_by_insur (
        subject_id int, 
        hadm_id int,
        icd9_code int,
        age float,
        gender text,
        admission_type text,
        insurance text,
        ethnicity text,
        hospital_expire_flag int,
        PRIMARY KEY ((insurance), subject_id, icd9_code)
    );

>>> INSERT INTO "RiaDSouza_de300lab4".patient_by_insur(subject_id, hadm_id, icd9_code, age, gender, admission_type, insurance, ethnicity, hospital_expire_flag)
VALUES (41976, 173269, 3891, 65.800000, 'M',	'EMERGENCY',	'Medicare',	'HISPANIC/LATINO - PUERTO RICAN',	0);

We want to create the table with the partition key being insurance, as we will primarily be using insurance type in the WHERE condition. We will also be grouping by prodecure, so we want to include icd9_code as a partition key. We use subject_id as the clsutering column since it is unique for each patient.

3. What proportion of the patients are above 55 with the procedure icd9_code between 3890 and 3899?

>>> CREATE TABLE IF NOT EXISTS "RiaDSouza_de300lab4".patient_by_age (
        subject_id int, 
        hadm_id int,
        icd9_code int,
        age float,
        gender text,
        admission_type text,
        insurance text,
        ethnicity text,
        hospital_expire_flag int,
        PRIMARY KEY ((age, icd9_code), subject_id)
    );

>>> INSERT INTO "RiaDSouza_de300lab4".patient_by_age (subject_id, hadm_id, icd9_code, age, gender, admission_type, insurance, ethnicity, hospital_expire_flag)
VALUES (41976, 173269, 3891, 65.800000, 'M',	'EMERGENCY',	'Medicare',	'HISPANIC/LATINO - PUERTO RICAN',	0);

We create this table with age and icd9_code as the partition keys, as we will be including both of these in the WHERE condition. We use subject_id as the clustering column as it is unique for each patient.